In [1]:
import pandas as pd

In [2]:
rfm_df  = pd.read_csv('RFM_Score.csv')
rfm_df

,Unnamed: 0,고객번호,Recency,Frequency,Monetary,Recency_R,Frequency_F,Monetary_M,RFM_Score
0,0,201812310749770,549 days,1,3988800,1.0,1.0,1,1.0
1,1,201812310749783,50 days,6,270539089,5.0,3.0,5,4.8
2,2,201812310749817,332 days,4,35573671,3.0,3.0,3,3.0
3,3,201812310749918,40 days,5,6152453,5.0,3.0,2,3.0
4,4,201812310749931,542 days,1,4579456,1.0,1.0,1,1.0
...,...,...,...,...,...,...,...,...,...
77557,77557,201912181051547,63 days,2,63304360,5.0,2.0,4,4.1
77558,77558,201912181051548,216 days,2,83928936,4.0,2.0,5,4.4
77559,77559,201912181051549,255 days,1,2076360,4.0,1.0,1,1.9
77560,77560,201912181051577,255 days,1,5310045,4.0,1.0,2,2.5


In [5]:
cond1 = (rfm_df['RFM_Score'] <= rfm_df['RFM_Score'].quantile(0.2))
cond2 = (rfm_df['RFM_Score'] <= rfm_df['RFM_Score'].quantile(0.4))
cond3 = (rfm_df['RFM_Score'] <= rfm_df['RFM_Score'].quantile(0.6))
cond4 = (rfm_df['RFM_Score'] <= rfm_df['RFM_Score'].quantile(0.8))

rfm_df['RFM_고객등급'] = 'A'
rfm_df.loc[cond4,'RFM_고객등급'] = 'B'
rfm_df.loc[cond3,'RFM_고객등급'] = 'C'
rfm_df.loc[cond2,'RFM_고객등급'] = 'D'
rfm_df.loc[cond1,'RFM_고객등급'] = 'E'

rfm_df['RFM_고객등급'].value_counts() / len(rfm_df)

E    0.252714
B    0.205358
A    0.194322
D    0.193845
C    0.153761
Name: RFM_고객등급, dtype: float64

In [6]:
rfm_df.rename(columns={'RFM_고객등급':'RFM_Class'}, inplace=True)

In [7]:
## 클래스별 고객 수 계산
temp_rfm_score1 = rfm_df.groupby('RFM_Class')['고객번호'].count().reset_index().rename(columns={'고객번호':'Count'})
 
## 클래스별 구매금액(매출)계산
temp_rfm_score2 = rfm_df.groupby('RFM_Class')['Monetary'].sum().reset_index()

# ## 클래스별 매출 기여도 계산
total_amount_of_sales = rfm_df['Monetary'].sum()
temp_rfm_score2['Monetary'] = temp_rfm_score2['Monetary'].map(lambda x : x/total_amount_of_sales)
 
## 데이터 결합
result_df = pd.merge(temp_rfm_score1,temp_rfm_score2,how='left',on=('RFM_Class'))

In [8]:
result_df

,RFM_Class,Count,Monetary
0,A,15072,0.645341
1,B,15928,0.216428
2,C,11926,0.070237
3,D,15035,0.050009
4,E,19601,0.017985


In [25]:
cond1 = rfm_df['RFM_Class']=='A'
A_list = rfm_df.loc[cond1]['고객번호'].tolist()
cond2 = rfm_df['RFM_Class']=='B'
B_list = rfm_df.loc[cond2]['고객번호'].tolist()
cond3 = rfm_df['RFM_Class']=='C'
C_list = rfm_df.loc[cond3]['고객번호'].tolist()
cond4 = rfm_df['RFM_Class']=='D'
D_list = rfm_df.loc[cond4]['고객번호'].tolist()
cond1 = rfm_df['RFM_Class']=='E'
E_list = rfm_df.loc[cond5]['고객번호'].tolist()

In [26]:
smp = pd.read_csv('sales_member_product_join(수정).csv')

/var/folders/sv/4hcm2v0d5hl630v663cqcpkh0000gn/T/ipykernel_9536/345215159.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  smp = pd.read_csv('sales_member_product_join(수정).csv')


In [27]:
cond1 = smp['고객번호'].isin(A_list)
cond2 = smp['고객번호'].isin(B_list)
cond3 = smp['고객번호'].isin(C_list)
cond4 = smp['고객번호'].isin(D_list)
cond5 = smp['고객번호'].isin(E_list)

smp.loc[cond1,'RFM_Class'] = 'A'
smp.loc[cond2,'RFM_Class'] = 'B'
smp.loc[cond3,'RFM_Class'] = 'C'
smp.loc[cond4,'RFM_Class'] = 'D'
smp.loc[cond5,'RFM_Class'] = 'E'

smp['RFM_Class'].value_counts()

A    69894
B    27608
E    20379
D    15297
C    14949
Name: RFM_Class, dtype: int64

In [28]:
smp.dropna(inplace=True)

In [30]:
cond1 = smp['RFM_Class']=='A'
cond2 = smp['RFM_Class']=='B'
cond3 = smp['RFM_Class']=='C'
cond4 = smp['RFM_Class']=='D'
cond5 = smp['RFM_Class']=='E'

# RFM 등급으로 나눠서 데이터 보기
smp_A = smp.loc[cond1]
smp_B = smp.loc[cond2]
smp_C = smp.loc[cond3]
smp_D = smp.loc[cond4]
smp_E = smp.loc[cond5]